In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer #load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler #스케일러
from sklearn.ensemble import AdaBoostClassifier #AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier #DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc #매트릭 측정
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import scipy.stats as stats

!pip install openpyxl
import pandas as pd

# 데이터 불러오기 (GTD 데이터셋)
df = pd.read_excel("./Full_GTD_Dataset_Full_Version.xlsx", engine='openpyxl')

# 내전 발생 지역 리스트
civil_war_countries = ["Syria", "Iraq", "Afghanistan", "Yemen", "Libya", "Somalia"]

# 내전 발생 지역 여부 컬럼 추가
df["civil_war_zone"] = df["country_txt"].apply(lambda x: 1 if x in civil_war_countries else 0)

# 테러 유형 필터링 (무장 공격: 'Armed Assault', 폭발/폭격: 'Bombing/Explosion')
target_types = ["Armed Assault", "Bombing/Explosion"]

# 내전 지역과 비내전 지역에서 테러 유형 빈도 계산
contingency_table = pd.crosstab(df["civil_war_zone"], df["attacktype1_txt"].isin(target_types))

# 카이제곱 검정 수행
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# 결과 출력
print("카이제곱 통계량:", chi2)
print("p-value:", p)

# p-value < 0.05이면 유의미한 차이가 있음 (즉, 내전 지역에서 해당 테러 유형 비율이 높을 가능성)
if p < 0.05:
    print("내전 지역과 비내전 지역에서 테러 유형 분포 차이가 통계적으로 유의미함.")
else:
    print("내전 지역과 비내전 지역의 테러 유형 분포 차이가 유의미하지 않음.")

카이제곱 통계량: 574.8790242856658
p-value: 4.875348093987522e-127
내전 지역과 비내전 지역에서 테러 유형 분포 차이가 통계적으로 유의미함.


In [ ]:
X = df[['iyear', 'civil_war_zone', 'weaptype1', 'nkill','nwound']]
y = df["attacktype1_txt"]

In [ ]:
X['nkill_missing'] = X['nkill'].isna().astype(int)
X['nwound_missing'] = X['nwound'].isna().astype(int)

<ipython-input-13-7dac61668538>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['nkill_missing'] = X['nkill'].isna().astype(int)
<ipython-input-13-7dac61668538>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['nwound_missing'] = X['nwound'].isna().astype(int)


In [ ]:
X['nkill'] = X['nkill'].fillna(-1)
X['nwound'] = X['nwound'].fillna(-1)

<ipython-input-26-02fe2b20a750>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['nkill'] = X['nkill'].fillna(-1)
<ipython-input-26-02fe2b20a750>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['nwound'] = X['nwound'].fillna(-1)


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209706 entries, 0 to 209705
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   iyear           209706 non-null  int64  
 1   civil_war_zone  209706 non-null  int64  
 2   weaptype1       209706 non-null  int64  
 3   nkill           209706 non-null  float64
 4   nwound          209706 non-null  float64
 5   nkill_missing   209706 non-null  int64  
 6   nwound_missing  209706 non-null  int64  
dtypes: float64(2), int64(5)
memory usage: 11.2 MB


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 209706 entries, 0 to 209705
Series name: attacktype1_txt
Non-Null Count   Dtype 
--------------   ----- 
209706 non-null  object
dtypes: object(1)
memory usage: 1.6+ MB


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
base_estimator = DecisionTreeClassifier(max_depth=1)

adaboost = AdaBoostClassifier(estimator=base_estimator, n_estimators=50, random_state=42)
adaboost.fit(X_train_scaled, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   random_state=42)

예측

In [ ]:
y_pred = adaboost.predict(X_test_scaled)

모델 평가

In [ ]:
accuracy_score(y_test, y_pred)

0.7659148347718278